In [4]:
import fitz
import requests
import io
import pickle
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import re
from bs4 import BeautifulSoup
import pandas as pd
import nltk

In [5]:
resp = requests.get("https://scriptum.cz")
soup = BeautifulSoup(resp.content)
sidebar = soup.find("div", {"id": "sidebar"})
print(sidebar.prettify())

<div id="sidebar">
 <ul>
  <li>
   <a href="/cs/stranky/s-a-m-i-z-d-a-t">
    S A M I Z D A T
   </a>
   <ul>
    <li>
     <a href="/cs/periodika/litva">
      LITVA
     </a>
     <ul>
      <li>
       <a href="/cs/periodika/ausra">
        Aušra
       </a>
      </li>
      <li>
       <a href="/cs/periodika/kronika-katolicke-cirkve-na-litve">
        Kronika katolické církve na Litvě
       </a>
      </li>
     </ul>
    </li>
    <li>
     <a href="/cs/periodika/polsko">
      POLSKO
     </a>
     <ul>
      <li>
       <a href="/cs/periodika/biuletyn-informacyjny-solidarnosci-polsko-czechoslowackiej">
        Biuletyn informac. solidarnosci polsko-czechoslowackiej
       </a>
      </li>
      <li>
       <a href="/cs/periodika/kryl">
        Zpěvník Krylových písní Wroclav ´89
       </a>
      </li>
     </ul>
    </li>
    <li>
     <a href="/cs/periodika/rusko">
      RUSKO
     </a>
     <ul>
      <li>
       <a href="/cs/periodika/antologia-samizdata-antologiya-samizda

In [7]:
# get hrefs for all samizdat periodicals
li_tags = sidebar.ul.find_all('li', recursive=False)
sidebar_samizdat = li_tags[0]
samizdat_hrefs = [li.a.get("href") for li in sidebar_samizdat.ul.find_all('li', recursive=False)][5:]
samizdat_hrefs[:10]

['/cs/periodika/ps-psi',
 '/cs/periodika/abecedni-casopis',
 '/cs/periodika/acta-incognitorum-eruditorum',
 '/cs/periodika/agentura-beat-nebeat-zved',
 '/cs/periodika/alternativa',
 '/cs/periodika/box',
 '/cs/periodika/brnensky-informacni-bulletin-nms',
 '/cs/periodika/bulletin-nezavisleho-miroveho-sdruzeni-nms',
 '/cs/periodika/bulletin-nms-v-liberci',
 '/cs/periodika/bulletin-sdruzeni-tgm']

In [8]:
# get hrefs for all exil periodicals
sidebar_exil = li_tags[1]
exil_hrefs = [li.a.get("href") for li in sidebar_exil.ul.find_all('li', recursive=False)][3:]
exil_hrefs[:10]

['/cs/periodika/150-000-slov',
 '/cs/periodika/academia-christiana',
 '/cs/periodika/acpr-in-germany',
 '/cs/periodika/acta',
 '/cs/periodika/afgansky-zapisnik',
 '/cs/periodika/agrarni-politika',
 '/cs/periodika/agrarni-vestnik',
 '/cs/periodika/ahoj',
 '/cs/periodika/akord-gries',
 '/cs/periodika/american-bulletin-the']

In [9]:
base_url = "https://scriptum.cz"

In [10]:
# test with one journal
resp = requests.get(base_url + samizdat_hrefs[29])
resp

<Response [200]>

In [11]:
soup = BeautifulSoup(resp.content)
title = soup.h1.get_text
basic_info = soup.h3.get_text()
a_tags = soup.find_all(lambda tag: tag.name == 'a' and 'ocr.pdf' in tag.get('href', ''))
hrefs = [tag.get('href') for tag in a_tags]
hrefs

['https://files.scriptum.cz/scriptum/ekologicky-bulletin/ekologicky-bulletin_1987_01_ocr.pdf',
 'https://files.scriptum.cz/scriptum/ekologicky-bulletin/ekologicky-bulletin_1988_02_ocr.pdf',
 'https://files.scriptum.cz/scriptum/ekologicky-bulletin/ekologicky-bulletin_1988_03_ocr.pdf',
 'https://files.scriptum.cz/scriptum/ekologicky-bulletin/ekologicky-bulletin_1988_04_ocr.pdf',
 'https://files.scriptum.cz/scriptum/ekologicky-bulletin/ekologicky-bulletin_1988_05_ocr.pdf',
 'https://files.scriptum.cz/scriptum/ekologicky-bulletin/ekologicky-bulletin_1988_06_ocr.pdf',
 'https://files.scriptum.cz/scriptum/ekologicky-bulletin/ekologicky-bulletin_1988_07_ocr.pdf',
 'https://files.scriptum.cz/scriptum/ekologicky-bulletin/ekologicky-bulletin_1988_08_ocr.pdf',
 'https://files.scriptum.cz/scriptum/ekologicky-bulletin/ekologicky-bulletin_1989_09_ocr.pdf',
 'https://files.scriptum.cz/scriptum/ekologicky-bulletin/ekologicky-bulletin_1989_10_ocr.pdf',
 'https://files.scriptum.cz/scriptum/ekologicky-bu

In [12]:
%%time
files_data = []
for periodical_href in samizdat_hrefs:
    try:
        resp = requests.get(base_url + periodical_href)
        soup = BeautifulSoup(resp.content)
        soup_content = soup.find("div", {"id" : "content"})
        periodal_title = soup_content.h1.get_text()
        periodical_basic_info = soup_content.h3.get_text()
        periodical_basic_info = periodical_basic_info.replace("\u00A0", "")
        periodical_basic_info = re.sub(r"\s+", " ", periodical_basic_info)
        periodical_basic_info = periodical_basic_info.split("*")
        a_tags = soup_content.find_all(lambda tag: tag.name == 'a' and 'ocr.pdf' in tag.get('href', ''))
        files_hrefs = [tag.get('href') for tag in a_tags]
        for href in files_hrefs:
            data = {"periodical_category" : "samizdat",
                    "periodical_href" : periodical_href,
                    "periodical_title" : periodal_title,
                    "periodical_basic_info" : periodical_basic_info}
            data["file_url"] = href
            files_data.append(data)
    except:
        pass
for periodical_href in exil_hrefs:
    try:
        resp = requests.get(base_url + periodical_href)
        soup = BeautifulSoup(resp.content)
        soup_content = soup.find("div", {"id" : "content"})
        periodal_title = soup_content.h1.get_text()
        periodical_basic_info = soup_content.h3.get_text()
        periodical_basic_info = periodical_basic_info.replace("\u00A0", "")
        periodical_basic_info = re.sub(r"\s+", " ", periodical_basic_info)
        periodical_basic_info = periodical_basic_info.split("*")
        a_tags = soup_content.find_all(lambda tag: tag.name == 'a' and 'ocr.pdf' in tag.get('href', ''))
        files_hrefs = [tag.get('href') for tag in a_tags]
        for href in files_hrefs:
            data = {"periodical_category" : "exil",
                    "periodical_href" : periodical_href,
                    "periodical_title" : periodal_title,
                    "periodical_basic_info" : periodical_basic_info}
            data["file_url"] = href
            files_data.append(data)
    except:
        pass

<unknown>:10: SyntaxWarning: invalid escape sequence '\s'
<unknown>:31: SyntaxWarning: invalid escape sequence '\s'


CPU times: user 33.1 s, sys: 317 ms, total: 33.4 s
Wall time: 1min 4s


In [176]:
files_df = pd.DataFrame(files_data)

In [178]:
len(files_df)

11617

In [181]:
files_df.sample(10)

,periodical_category,periodical_href,periodical_title,periodical_basic_info,file_url
11259,exil,/cs/periodika/zpravodaj_curych,Zpravodaj Curych,"[, Curych , 1968 – , týdně, od 1969 čtrnáct...",https://files.scriptum.cz/scriptum/zpravodaj_c...
2165,samizdat,/cs/periodika/ze-zasuvky-i-z-bloku,Ze zásuvky i z bloku,"[, [Praha] , 1984 – 1989 , nepravidelně , ]",https://files.scriptum.cz/scriptum/%5Bnode%5D/...
6479,exil,/cs/periodika/kulturni-klub,Kulturní klub,"[, Vídeň , 1981 - , měsíčně , ]",https://files.scriptum.cz/scriptum/kulturni-kl...
3584,exil,/cs/periodika/bic,Bič,"[, Gagny (Francie) , 1959 - 1990 , měsíčně , ]",https://files.scriptum.cz/scriptum/bic/bic_198...
1331,samizdat,/cs/periodika/res-publica,Res publica,"[, Šumperk , 1973 – 1975; 1987 – 1989; 1991 ...",https://files.scriptum.cz/scriptum/%5Bnode%5D/...
7475,exil,/cs/periodika/narodni-politika-69-80,Národní politika,"[, Mnichov , 1969 - 1992 , měsíčně , ]",https://files.scriptum.cz/scriptum/narodni-pol...
6649,exil,/cs/periodika/kus-reci,Kus řeči,"[, Champigny , 1982 - ? , nepravidelně , ]",https://files.scriptum.cz/scriptum/%5Bnode%5D/...
9252,exil,/cs/periodika/severske-listy,Severské listy,"[, Åtvidaberg, od 1987 Uppsala (Švédsko) , 1...",https://files.scriptum.cz/scriptum/severske-li...
1484,samizdat,/cs/periodika/res-publica,Res publica,"[, Šumperk , 1973 – 1975; 1987 – 1989; 1991 ...",https://files.scriptum.cz/scriptum/%5Bnode%5D/...
2959,exil,/cs/periodika/americke-listy,Americké listy,"[, New York , 1962 - 1989 , týdně , ]",https://files.scriptum.cz/scriptum/americke-li...


In [179]:
files_df.to_json("../data/files_df.json")

In [9]:
files_df = pd.read_json("../data/files_df.json")
len(files_df)

11617

In [10]:
h4_tags = [h4 for h4 in soup.find_all('h4') if re.search("\d{4}", h4.get_text())]

# For each `h4` tag, find the `p` tag just after it and get all `a` tags within
for h4 in h4_tags:
    p = h4.find_next_sibling('p')

    if p is not None:
        a_tags = p.find_all('a')

        # Extract hrefs
        hrefs = [a.get('href') for a in a_tags]

        for href in hrefs:
            print(href)

In [19]:
def get_cleaned_text(url):
    try:
        pdf_object = io.BytesIO(requests.get(url).content)
        doc = fitz.open("pdf", pdf_object.read())
        pages_n = len(doc)
        text = ""
        for n, p in enumerate(doc):
            text += p.get_text() + "[pageend{}]".format(str(n + 1))
        #pattern = "(_+)?(\n?Studie[\s_]+\d{4}\n?)(\W*\w{0,3}\s*/\s*\d\n?)?"
        #text = re.sub(pattern, "\n", text)
        #text = re.sub("\n\s?\d{1,3}\s?\n", "\n", text)
        text = re.sub("\xad\n", "", text)
        text = re.sub("\s\s+", " ", text.replace("\n", " "))
        text = text.replace("- \n", "")
        #year = int(re.search("\d{4}", filename).group())
        return text, pages_n
    except:
        return "", 0

In [13]:
url = (files_df["file_url"][20])
url

'https://files.scriptum.cz/scriptum/acta-incognitorum-eruditorum/acta-incognitorum_1980-81_r06_01-05_ocr.pdf'

In [14]:
pdf_object = io.BytesIO(requests.get(url).content)
doc = fitz.open("pdf", pdf_object.read())

In [15]:
len(doc)

310

In [17]:
text = ""
for n, p in enumerate(doc):
    text += p.get_text() + "[pageend{}]".format(str(n + 1))

In [18]:
text

'ACTA INCOGNITORUM VI, 1980-81, 1-5\n[pageend1]číslo ročník měsíc rok\nACTA INCOGNITORUM\n1  VI  9 80\n[pageend2]REFERÁTY A STUDIE\n[pageend3]3\nR E C E S E\nEva Cieslarová\n"Recese jako hnutí patří už historii. Lidí prostě \n7-onají v určitém čase a prostoru. íyš enka r eese je \nS\' aozřcjmě vččnái nejen prot, že lidi budou vždycky \nkýtnt, ale především ppoto, že b í žádá všechno to kolem« \nReceeo nebyla totiž nikdy moc státotvorná. Vznikla jako \n3 ?-ce proti něčemu. Pi OH autorite, proti různý/u těm \n"nesmíš" rodiny, školy a moci a taky proti té době, \nkterá tenkrát př d ”ál’-ou v dla všechny neodvratně... \nČlověk by málem řekl do Bodele*\' ... Recesisté se snažili \nvyužít každé situace k útoku na své okolí, učinit z všed\xad\nních životních situací hru. ?řitom si přišlo na své i \nokolí, ha které se útočilo právě tí , že bylo zatahováno \ndo liry."\nTolik Vladimír Bor, jeden ze zakladatel recesistic\xad\nkého hnutí.\nAle vr-íms se raději na začátek, do raných třicátých \nlet

In [187]:
os.mkdir("../data/rawtexts/")

In [198]:
path = "../data/rawtexts/"
len(os.listdir(path))

31

In [20]:
def extract_and_save_text(url):
    path = "../data/rawtexts/"
    filename = url.replace(".pdf", ".txt").rpartition("/")[2]
    ready_files = os.listdir(path)
    if len(ready_files) in range(0,len(files_df), 100):
        print(len(ready_files))
    if filename not in ready_files:
        text, pages_n = get_cleaned_text(url)
        with open(path + filename, 'w') as f:
            f.write(text)
        return pages_n

In [21]:
# test with sample
files_df.sample(10)["file_url"].apply(extract_and_save_text)

9088     16
7346      8
44       10
11114    52
2839      8
8731      0
1667      0
9487      8
6391     24
2269      1
Name: file_url, dtype: int64

In [22]:
%%time
files_df["pages_n"] = files_df["file_url"].apply(extract_and_save_text)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
CPU times: user 30min 18s, sys: 6min 42s, total: 37min 1s
Wall time: 2h 3min 15s


In [23]:
files_df["pages_n"].sum()

318091.0

In [27]:
path = "../data/rawtexts/"
filenames = os.listdir(path)
len(filenames)

11615

In [52]:
filename = filenames[10000]
with open(path + filename) as f:
    text = f.read()
print(text[:10000])

Zpravodaj Čechů a Slováků ve Švýcarsku Sprayodaj Slovákov a Čechov vo Švajčiarsku Jubilant 11 ročník 40 / 2007 Oskar Leo Krause Snad jsem nezabloudil... Pamatujete se ještě na tento plakát? Oskar Krause jej zvolil pro titulní stranu své nové publikace. Pozvánku k jejímu křtu najdete v rubrice Nové publikace na str. 30 [pageend1]NOVINKA: ChceŤe cestovat nebo pozvat Vaše milé na dovolenou s Čedokem Praha za české ko- runy? Nabízíme kompletní nabídku zájezdů do celého světa ze všech katalogů českého Čedoku Advent: bu s ČSAD + 2 noci, hotel *** + snídaně, okruh městem, 2 jízdenky na MHD - cena již od Fr 214. - Letadlem Curych, Basilej, Ženeva - 2 noci hotel *** + snídaně, okruh městem, 2 jízdenky MHĎ. Cena již od Fr 436^-______ CSA+Swiss Zürich-Praha a zpět již od Fr 166.- Výhodné letecké spojení ze Ženevy a Basileje do Prahy se Swissem nebo Easyjetem - přímá linka Denně s ČSAD-Tišnov Lucern-Curych-Plzeň-Praha-Brno-Bratislava__________ EUROBUS z Curychu do RAKOUSKA a na SLOVENSKO (denně do

In [53]:
text.count("[pageend")

52

In [54]:
%%time
doc = nlp(text)

CPU times: user 34.1 s, sys: 6.88 s, total: 41 s
Wall time: 29.1 s


In [56]:
lemmatized_sents = []
for sent in doc.sentences:
    lemmatized_sents.append([t.lemma for t in sent.words if t.upos in ["PROPN", "NOUN", "VERB", "ADJ"]])
print(lemmatized_sents[100:110])

[['dát', 'vytvářet', 'spolek', 'nový', 'spolek', 'český', 'kultura', 'čtenářský', 'klub', 'aj'], ['funkcionář', 'spolek', 'svaz', 'domnívat', 'současný', 'anketa', 'namířený', 'spolek'], ['chtít', 'vysvětlení', 'vyvrátit'], ['obdivovat', 'chválit', 'angažmá', 'práce', 'funkcionář', 'obětovat', 'organizace', 'spolkový', 'činnost', 'hodina', 'den', 'týden', 'měsíc', 'odměňovaný'], ['radostný', 'výsledek', 'pomíjivý'], ['osobnost', 'jít'], ['hobby', 'život', 'zvolit'], ['odměna', 'uspokojení', 'dát', 'pohyb', 'klapat'], ['šikovný', 'definovat', 'projekt', 'publikum', 'zapojovat', 'sponzor', 'jít', 'veřejnost', 'budit', 'zájem', 'ponoukat', 'příliv', 'finance'], ['cizokrajný', 'spolkyjsa', 'nucený', 'vyvíjet', 'domácí', 'prostředí', 'odpovídající', 'šarm', 'těžký', 'handicapovaný']]


In [ ]:
os.mkdir("../data/lemsents/")

In [58]:
text = "text na první stránce[pageend1]na druhé stránce[pageend455]"
text = re.sub("\[pageend\d+\]", " ", text)
text

'text na první stránce na druhé stránce '

In [75]:
processed = []
def get_lemmatized_sentences(filename):
    with open(path + filename) as f:
        text = f.read()
    if filename not in []: # "studie_1990_132_ocr.pdf"]:
        ready = os.listdir("../data/lemsents/")
        if filename not in ready:
            text = re.sub("\[pageend\d+\]", " ", text)
            chunks = re.findall(r'.{0,50000}\.\s', text, re.DOTALL)
            chunks_end = sum([len(chunk) for chunk in chunks])
            chunks.append(text[chunks_end:])
            docs = [nlp(chunk) for chunk in chunks]
            lemmatized_sents = []
            for doc in docs:
                for sent in doc.sentences:
                    lemmatized_sents.append(" ".join([t.lemma for t in sent.words if t.upos in ["PROPN", "NOUN", "VERB", "ADJ"]]) + "\n")
            processed.append(filename)
            pathfn = "../data/lemsents/" + filename
            with open(pathfn, 'w') as f:
                f.writelines(lemmatized_sents)
            if len(ready) in range(0, len(filenames), 100):
                print(len(ready))

In [77]:
get_lemmatized_sentences(filenames[0])

In [81]:
with open("../data/lemsents/" + filenames[0], "r") as f:
    sents = f.readlines()

In [82]:
sents[:10]

['USPS americký list\n',
 'jediný československý týdeník východ spojený stát Če skupeň slo e n skupeň rok ic kůň t e n ík e c hodina slo rok w e e litr yorský list\n',
 'československý mírový hnutí\n',
 'poslat\n',
 'měsíčník československý svět mít prosit mysl\n',
 'předplatný obdarovat mít číslo\n',
 'Hana Adamcová potomek dělnický prokurátor holič Zižkův dotáhnout generální prokuratura\n',
 'Hana Adamcová strana vylévat bohatý poznatek různý forma boj mír vupůjčovat slovník znít vSe úžas dočítat pravda Československo jiný socialistický země zavírat důsledný boj mír běžný západní země vyhazovat práce\n',
 'mírový úsilí lid komunista příslušník ostatní politický strana nestraník člen odborový mládežnický ženský jiný organizace věřící nejmense rozpor státní politika\n',
 'druhý strana ovso soudružka mluvit čistý pravda pravit najít prostý občan hlava vyjmenovat krok vláda CSSR národní mezinárodní měřítko udělat zájem míra poslední rok jmenovat rezoluce ÚVKSC\n']

In [83]:
for filename in filenames:
    get_lemmatized_sentences(filename)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300


TypeError: sequence item 6: expected str instance, NoneType found

In [ ]:
%%time
#scriptum_df["lemmatized_sents"] = scriptum_df.apply(lambda row: get_lemmatized_sentences(row["filename"], row["text"], row["n_chars"]), axis=1)
# scriptum_df.to_json("../data/large_files/scriptum_df_lemmata.json")
scriptum_df = pd.read_json("../data/large_files/scriptum_df_lemmata.json")
scriptum_df.head(5)

### Nejčastější slova po obdobích

In [ ]:
periods_freqs = {}
periods = [(1958,1968), (1969,1976), (1977,1990)]
periods_labels = ["Studie {0}-{1}".format(str(period[0]), str(period[1])) for period in periods]
for period, period_label in zip(periods, periods_labels):
    subset_df = scriptum_df[scriptum_df["year"].between(period[0], period[1])]
    lemmatized_sents = [sentences for sentences in subset_df["lemmatized_sents"] if sentences != None]
    sentences_flat = [sent for file_sents in lemmatized_sents for sent in file_sents]
    lemmata_list = [lemma for sent in sentences_flat for lemma in sent]
    lemmata_list = [lemma for lemma in lemmata_list if len(lemma) > 1]
    lemmata_freqs = nltk.FreqDist(lemmata_list).most_common()
    periods_freqs[period_label] = lemmata_freqs

Podívejme se namátkou na 50 nejčastějších slov z nejranějšího období.

In [ ]:
periods_freqs[periods_labels[0]][:50]

In [ ]:
wc = WordCloud(width=800, height=400).generate_from_frequencies(dict(periods_freqs[periods_labels[0]][:50]))
plt.imshow(wc, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
n = 100
fig, axs = plt.subplots(3,1, figsize=(4.5, 5) , dpi=300, tight_layout=True)
for item, ax in zip(periods_freqs.items(), axs.ravel()):    
    wc = WordCloud(width=800, height=400, background_color="white").generate_from_frequencies(dict(item[1][:n]))
    ax.imshow(wc, interpolation='bilinear')
    ax.set_title(item[0])
    ax.axis("off")

### Lematizované věty s vybranými slovy

Nyní si nadefinujeme funkci, pomocí které budeme moci vyextrahovat veškeré věty obsahující konkrétní slova.

In [ ]:
def extract_target_sents(lemmatized_sents, targets):
    try:
        return [sent for sent in lemmatized_sents if any(target in sent for target in targets)]
    except:
        return []

In [ ]:
my_targets = ["kultura"]
scriptum_df["target_sents"] = scriptum_df["lemmatized_sents"].apply(extract_target_sents, args=(my_targets,))

In [ ]:
scriptum_df["target_sents"].apply(lambda x: len(x) if x != None else 0).sum()

In [ ]:
scriptum_df["target_sents"]

Jaká slova se v těchto větách objevují nejčastěji?

In [ ]:
column_sents = scriptum_df["target_sents"].tolist()
target_sents_counts = nltk.FreqDist([t for sent in [t for sent in column_sents for t in sent] for t in sent]) #
target_sents_counts = target_sents_counts.most_common()
target_sents_counts[:20]

In [ ]:
wc = WordCloud(width=800, height=400, background_color="white").generate_from_frequencies(dict(target_sents_counts[:50]))
plt.imshow(wc) # , interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
column_sents = scriptum_df["target_sents"].tolist()
target_sents_counts = nltk.FreqDist([t for sent in [t for sent in column_sents for t in sent] for t in sent]) #
target_sents_counts = target_sents_counts.most_common()
target_sents_counts[:20]